In [1]:
import matplotlib.pyplot as plt

In [2]:
from posteriordb import PosteriorDatabase

pdb_path = "/workspaces/jupyter-data/posteriordb/posterior_database/"
my_pdb = PosteriorDatabase(pdb_path)
model_name = "diamonds"
model = my_pdb.model(model_name)
data = my_pdb.data(model_name)

In [4]:
print(model.stan_code())

// generated with brms 2.10.0

functions {
}
data {
  int<lower=1> N;  // number of observations
  vector[N] Y;  // response variable
  int<lower=1> K;  // number of population-level effects
  matrix[N, K] X;  // population-level design matrix
  int prior_only;  // should the likelihood be ignored?
}
transformed data {
  int Kc = K - 1;
  matrix[N, Kc] Xc;  // centered version of X without an intercept
  vector[Kc] means_X;  // column means of X before centering
  for (i in 2:K) {
    means_X[i - 1] = mean(X[, i]);
    Xc[, i - 1] = X[, i] - means_X[i - 1];
  }
}
parameters {
  vector[Kc] b;  // population-level effects
  // temporary intercept for centered predictors
  real Intercept;
  real<lower=0> sigma;  // residual SD
}
transformed parameters {
}
model {
  // priors including all constants
  target += normal_lpdf(b | 0, 1);
  target += student_t_lpdf(Intercept | 3, 8, 10);
  target += student_t_lpdf(sigma | 3, 0, 10)
    - 1 * student_t_lccdf(0 | 3, 0, 10);
  // likelihood includ

In [3]:
import torch

N = torch.tensor(data.values()["N"])
k = torch.tensor(data.values()["K"])
y = torch.tensor(data.values()["Y"])
x = (
    torch.tensor(data.values()["X"]) - torch.tensor(data.values()["X"]).mean(dim=0)
)[:, 1:]

In [4]:
def p(alpha, beta, sigma):
    return (
        dist.Independent(dist.Normal(torch.zeros(k-1), torch.ones(k-1)), 1).log_prob(beta) +
        dist.StudentT(3, 8, 10).log_prob(alpha) +
        dist.StudentT(3, 0, 10).log_prob(sigma) - torch.log(torch.tensor(0.5)) +
        dist.Independent(dist.Normal((alpha + x @ beta.T).T, sigma.unsqueeze(1)), 1).log_prob(y)
    )

In [7]:
import torch
import torch.distributions as dist
import flowtorch.bijectors
import flowtorch.params

flow = flowtorch.bijectors.AffineAutoregressive(
    flowtorch.params.DenseAutoregressive(
        hidden_dims=(32,32,32)
    )
)
base_dist = dist.Normal(torch.zeros(k+2), torch.ones(k+2))
new_dist, params = flow(base_dist)

In [8]:
import torch.optim as optim
import torch.nn as nn
from tqdm.auto import tqdm

optimizer = optim.Adam(params.parameters(), lr=1e-2)

n = 1000
for _ in tqdm(range(int(1e3))):
    optimizer.zero_grad()
    samples = new_dist.rsample((n,))
    log_q = new_dist.log_prob(samples)

    alpha = samples[:,0]
    beta = samples[:,1:k]
    sigma = samples[:,k].exp()
    log_p = p(alpha, beta, sigma) #+ samples[:,k] * sigma
    loss = (log_q - log_p).mean()
    loss.backward(retain_graph=True)
    optimizer.step()

    marginal_lik = torch.logsumexp(log_p - log_q - torch.log(torch.tensor([n])), dim=0)
    tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}", end='')

ELBO: 2744.085205078125, log p(y): 2855.183349609375555


In [6]:
import torch
import torch.distributions as dist
import flowtorch.bijectors
import flowtorch.params
import torch.optim as optim
import torch.nn as nn
from tqdm.auto import tqdm

data = []

for trial in range(100):
    for num_layers in range(1,5):
        flow = flowtorch.bijectors.AffineAutoregressive(
            flowtorch.params.DenseAutoregressive(
                hidden_dims=(32,)*num_layers
            )
        )
        base_dist = dist.Normal(torch.zeros(k+2), torch.ones(k+2))
        new_dist, params = flow(base_dist)
        optimizer = optim.Adam(params.parameters(), lr=1e-2)

        n = 1000
        for _ in tqdm(range(int(1e3))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            if torch.isnan(samples).any(): continue

            log_q = new_dist.log_prob(samples)

            alpha = samples[:,0]
            beta = samples[:,1:k]
            sigma = samples[:,k].exp()
            log_p = p(alpha, beta, sigma) #+ samples[:,k] * sigma
            loss = (log_q - log_p).mean()
            loss.backward(retain_graph=True)
            optimizer.step()

            marginal_lik = torch.logsumexp(log_p - log_q - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}", end='')

        if marginal_lik < 2500: continue
        data.append({
            'method': 'ADVI',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })

        # TAF, train just DF
        dfs = nn.Parameter(data=torch.ones(1)*10)
        new_dist.base_dist = dist.StudentT(df=dfs*torch.ones(k+2), loc=torch.zeros(k+2), scale=torch.ones(k+2))
        optimizer = optim.Adam([dfs], lr=1e-2)
        for _ in tqdm(range(int(1e2))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            if torch.isnan(samples).any(): continue
            log_q = new_dist.log_prob(samples)

            alpha = samples[:,0]
            beta = samples[:,1:k]
            sigma = samples[:,k].exp()
            log_p = p(alpha, beta, sigma) #+ samples[:,k] * sigma
            loss = (log_q - log_p).mean()
            loss.backward(retain_graph=True)
            optimizer.step()

            marginal_lik = torch.logsumexp(log_p - log_q - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}", end='')

        data.append({
            'method': 'TAF',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })

        # TAF, train just DF
        dfs = nn.Parameter(data=dfs.data*torch.ones(k+2))
        new_dist.base_dist = dist.StudentT(df=dfs, loc=torch.zeros(k+2), scale=torch.ones(k+2))
        optimizer = optim.Adam([dfs], lr=1e-2)
        for _ in tqdm(range(int(1e2))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            if torch.isnan(samples).any(): continue
            log_q = new_dist.log_prob(samples)

            alpha = samples[:,0]
            beta = samples[:,1:k]
            sigma = samples[:,k].exp()
            log_p = p(alpha, beta, sigma) #+ samples[:,k] * sigma
            loss = (log_q - log_p).mean()
            loss.backward(retain_graph=True)
            optimizer.step()

            marginal_lik = torch.logsumexp(log_p - log_q - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}", end='')

        data.append({
            'method': 'ATAF',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })
        
pickle.dump(data, open("diamonds.pkl", "wb"))

ELBO: 856.3377685546875, log p(y): 2562.155517578125575


ELBO: 1004.0053100585938, log p(y): 2740.8449707031255


ELBO: 535.5269775390625, log p(y): 2807.1596679687555


ELBO: 2923.075927734375, log p(y): 2981.962158203125552


ELBO: 2904.009033203125, log p(y): 2979.973388671875


...

ELBO: 2590.58935546875, log p(y): 2816.9677734375875555


ELBO: 2498.173095703125, log p(y): 2834.150878906255


ELBO: 2487.996337890625, log p(y): 2899.777099609375


ELBO: 2627.158935546875, log p(y): 2817.402832031255555


KeyboardInterrupt: 

In [27]:
import pickle
#pickle.dump(data, open("diamonds.pkl", "wb"))
data = pickle.load(open("diamonds.pkl", "rb"))

In [28]:
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

df = pd.DataFrame(data).melt(id_vars=['method','num_layers'])

In [29]:
df

,method,num_layers,variable,value
0,ADVI,1,elbo,856.337769
1,TAF,1,elbo,1004.005310
2,ATAF,1,elbo,535.526978
3,ADVI,2,elbo,2923.075928
4,TAF,2,elbo,2904.009033
...,...,...,...,...
61,TAF,4,log_p_y,2655.429199
62,ATAF,4,log_p_y,2635.206543
63,ADVI,1,log_p_y,2816.967773
64,TAF,1,log_p_y,2834.150879


In [30]:
df.groupby(['method', 'num_layers', 'variable']).describe()

value                                         \
                           count         mean          std          min   
method num_layers variable                                                
ADVI   1          elbo       3.0  2028.676392  1015.564747   856.337769   
                  log_p_y    3.0  2759.818278   176.182589  2562.155518   
       2          elbo       2.0  2873.900269    69.544884  2824.724609   
                  log_p_y    2.0  2969.734009    17.293215  2957.505859   
       3          elbo       3.0   -31.933512  4237.924677 -4923.806641   
                  log_p_y    3.0  2697.114421   103.870033  2611.117920   
       4          elbo       3.0   -68.549683  3749.547417 -4381.683594   
                  log_p_y    3.0  2626.064535   122.838011  2513.989746   
ATAF   1          elbo       3.0  1860.388143  1147.891732   535.526978   
                  log_p_y    3.0  2866.065511    51.191406  2807.159668   
       2          elbo       2.0  2842.751709    88.369703  2780.264893   
                  log_p_y    2.0  2976.751099     6.624465  2972.066895   
       3          elbo       3.0   -13.685547  4289.322499 -4963.328125   
                  log_p_y    3.0  2725.612549   119.086851  2645.759277   
       4          elbo       3.0   -64.149089  3902.725523 -4553.748535   
                  log_p_y    3.0  2714.039551   109.899924  2635.206543   
TAF    1          elbo       3.0  2027.492493   887.316561  1004.005310   
                  log_p_y    3.0  2822.723633    76.805271  2740.844971   
       2          elbo       2.0  2839.637207    91.035510  2775.265381   
                  log_p_y    2.0  2973.845947     8.665511  2967.718506   
       3          elbo       3.0   -32.627930  4334.053569 -5033.805664   
                  log_p_y    3.0  2758.631917   102.856110  2672.373291   
       4          elbo       3.0  -115.953410  4021.127915 -4744.854980   
                  log_p_y    3.0  2736.158854    82.794257  2655.429199   

                                                                                
                                    25%          50%          75%          max  
method num_layers variable                                                      
ADVI   1          elbo      1723.463562  2590.589355  2614.845703  2639.102051  
                  log_p_y   2689.561646  2816.967773  2858.649658  2900.331543  
       2          elbo      2849.312439  2873.900269  2898.488098  2923.075928  
                  log_p_y   2963.619934  2969.734009  2975.848083  2981.962158  
       3          elbo     -1310.102661  2303.601318  2414.003052  2524.404785  
                  log_p_y   2639.412964  2667.708008  2740.112671  2812.517334  
       4          elbo     -1310.205994  1761.271606  2088.017273  2414.762939  
                  log_p_y   2560.400757  2606.811768  2682.101929  2757.392090  
ATAF   1          elbo      1511.761658  2487.996338  2522.818726  2557.641113  
                  log_p_y   2849.209717  2891.259766  2895.518433  2899.777100  
       2          elbo      2811.508301  2842.751709  2873.995117  2905.238525  
                  log_p_y   2974.408997  2976.751099  2979.093201  2981.435303  
       3          elbo     -1328.662720  2306.002686  2461.135742  2616.268799  
                  log_p_y   2657.174316  2668.589355  2765.539185  2862.489014  
       4          elbo     -1355.286255  1843.176025  2180.650635  2518.125244  
                  log_p_y   2651.270264  2667.333984  2753.456055  2839.578125  
TAF    1          elbo      1751.089203  2498.173096  2539.236084  2580.299072  
                  log_p_y   2787.497925  2834.150879  2863.662964  2893.175049  
       2          elbo      2807.451294  2839.637207  2871.823120  2904.009033  
                  log_p_y   2970.782227  2973.845947  2976.909668  2979.973389  
       3          elbo     -1362.273315  2309.259033  2467.960938  2626.662842  
                  log_p_y   2701.715576  2731.057861  280

In [32]:
import numpy as np

gs = my_pdb.posterior("diamonds-diamonds").reference_draws()
gs_dict = {}
num_chains = len(gs)
num_samples = len(gs[0][next(iter(gs[0]))])

for i,chain in enumerate(gs):
    for var in chain:
        if '[' not in var:
            if var not in gs_dict:
                gs_dict[var] = np.zeros((num_chains, num_samples))
            gs_dict[var][i,:] = np.array(chain[var])
        else:
            name = var.split('[')[0]
            idx = int(var.split('[')[1].split(']')[0]) - 1
            if name not in gs_dict:
                var_size = len(list(filter(lambda x: x.startswith(name), chain)))
                gs_dict[name] = np.zeros((num_chains,num_samples,var_size))
            gs_dict[name][i,:,idx] = np.array(chain[var])
            
data = my_pdb.data(model_name)
N = torch.tensor(data.values()["N"])
k = torch.tensor(data.values()["K"])
y = torch.tensor(data.values()["Y"])
x = (
    torch.tensor(data.values()["X"]) - torch.tensor(data.values()["X"]).mean(dim=0)
)[:, 1:]

In [33]:
gs_dict.keys()

dict_keys(['b', 'Intercept', 'sigma'])

In [38]:
list(map(lambda x: x.shape, [
    torch.tensor(gs_dict['Intercept']),
    torch.tensor(gs_dict['b']),
    torch.tensor(gs_dict['sigma'])
]))

[torch.Size([10, 1000]), torch.Size([10, 1000, 24]), torch.Size([10, 1000])]

In [40]:
import torch.distributions as dist
            
#log_p = p(alpha, beta, sigma) #+ samples[:,k] * sigma
ps = p(
    torch.tensor(gs_dict['Intercept']).flatten(end_dim=1).float(),
    torch.tensor(gs_dict['b']).flatten(end_dim=1).float(),
    torch.tensor(gs_dict['sigma']).flatten(end_dim=1).float(),
)
print(ps.mean(), ps.std() / np.sqrt(ps.shape[0]))

tensor(3274.5872) tensor(0.0366)


In [42]:
y.shape

torch.Size([5000])

In [36]:
p_gs.shape

NameError: name 'mu' is not defined

In [51]:
torch.tensor(gs_dict['tau']).flatten(end_dim=1).outer(torch.ones(3)).shape

torch.Size([10000, 3])

In [38]:
gs_dict['tau'].shape

(10, 1000)

In [39]:
gs_dict['theta'].shape

(10, 1000, 8)